##### Sprintの目的
- アンサンブル学習について理解する

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

from decimal import Decimal, ROUND_HALF_UP

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error

#np.set_printoptions(threshold=0)

3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。


- ブレンディング
- バギング
- スタッキング

以前も利用した回帰のデータセットを用意します。


[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習用（train）8割、検証用（val）2割に分割してください。

In [2]:
original_data = pd.read_csv('train.csv')
original_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
X = original_data.loc[:, ["GrLivArea", "YearBuilt"]].values
display(X)
print(X.shape)

y = original_data["SalePrice"].values
display(y)
print(y.shape)

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=0)

array([[1710, 2003],
       [1262, 1976],
       [1786, 2001],
       ...,
       [2340, 1941],
       [1078, 1950],
       [1256, 1965]])

(1460, 2)


array([208500, 181500, 223500, ..., 266500, 142125, 147500])

(1460,)


/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

[sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)


[sklearn.svm.SVR — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)


[sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

### 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。


[sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)

#### 単一モデルでのMSE(平均二乗誤差)

In [11]:
#線形回帰
np.set_printoptions(threshold=0)

lr =  LinearRegression()
lr.fit(X_train, y_train)
print("予測値：\n", lr.predict(X_val))
print("MSE：", f"{mean_squared_error(y_val, lr.predict(X_val)):,.2f}")

予測値：
 [264908.90812295 155745.99630863 127984.53226316 ... 223602.56439896
  68715.66448996  87870.91973285]
MSE： 2,942,066,921.67


In [5]:
#SVM

svr = SVR()
svr.fit(X_train, y_train)
print("予測値：\n", svr.predict(X_val))
print("MSE：", f"{mean_squared_error(y_val, svr.predict(X_val)):,.2f}")

予測値：
 [162999.39962414 162999.31753914 162999.39962041 ... 162999.39962414
 162999.39962414 162999.39962412]
MSE： 7,243,319,908.93


In [66]:
#決定木

tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
print("予測値：\n", tree.predict(X_val))
print("MSE：", f"{mean_squared_error(y_val, tree.predict(X_val)):,.2f}")

予測値：
 [151400. 140000. 125000. ... 169000. 118000. 116900.]
MSE： 3,374,967,201.20


#### ブレンディングクラスの作成

In [7]:
class Blending_model():
    """
    ブレンディングのスクラッチ実装

    Parameters
    ----------
    
    """
    def __init__(self, estimators):
        self.estimators = estimators
        
    def fit(self, X, y):
        """
        N個の多様なモデルを独立して学習させ、辞書に保存する
        
        Parameters
        ----------
        X : ndarray(n_samples, n_features)
            訓練データの特徴量
        y : ndarray(n_samples, )
            正解データ
        estimators : [model1(), model2(), ...] 
            学習させるモデル
        
        """
        self.estimator_dic = {}
        
        for i, estimator in enumerate(self.estimators):
            model = estimator #モデルのインスタンス化
            self.estimator_dic[i] = model.fit(X, y) #fitさせたモデルを辞書に保存
        
        pass
    
    def predict(self, X):
        """
        辞書に保存された各学習モデルの予測値を算出後、その平均値を計算
        
        Parameters
        ----------
        X : ndarray(n_samples, n_features)
            検証データの特徴量
        
        Return
        ----------
        y_pred : ndarray(n_samples, )
            検証データの予測値
        """
        predict_list = []
        
        for _, model in self.estimator_dic.items():
            predict_list.append(model.predict(X))
        
        #print("predict_list:\n", predict_list)
        
        y_pred = np.mean(predict_list, axis=0)
        
        return y_pred

#### ブレンドモデルでのMSE

In [8]:
#検証1
#「線形回帰、SVM、決定木」のブレンド

bm = Blending_model(estimators=[LinearRegression(), SVR(), DecisionTreeRegressor()])
bm.fit(X_train, y_train)
print("予測値：\n", bm.predict(X_val))
print("---------")
print("線形回帰MSE：", f"{mean_squared_error(y_val, lr.predict(X_val)):,.2f}")
print("MSE：", f"{mean_squared_error(y_val, bm.predict(X_val)):,.2f}")

予測値：
 [193102.76924903 152981.77128259 138661.31062786 ... 185200.65467437
 116571.68803803 122590.10645233]
---------
線形回帰MSE： 2,942,066,921.67
MSE： 2,900,940,898.78


単一モデルで一番良かった線形回帰より、僅かに精度の良いモデルとなった。

In [9]:
#検証2
#「線形回帰、SVM」のブレンド

bm = Blending_model(estimators=[LinearRegression(), SVR()])
bm.fit(X_train, y_train)
print("予測値：\n", bm.predict(X_val))
print("---------")
print("線形回帰MSE：", f"{mean_squared_error(y_val, lr.predict(X_val)):,.2f}")
print("MSE：", f"{mean_squared_error(y_val, bm.predict(X_val)):,.2f}")

予測値：
 [213954.15387355 159372.65692389 145491.96594178 ... 193300.98201155
 115857.53205705 125435.15967849]
---------
線形回帰MSE： 2,942,066,921.67
MSE： 3,776,403,745.45


精度の悪いモデルと良いモデルをブレンドすると精度が悪くなることは想像しやすい。

In [10]:
#検証3
#「線形回帰、決定木」のブレンド

bm = Blending_model(estimators=[LinearRegression(), DecisionTreeRegressor()])
bm.fit(X_train, y_train)
print("予測値：\n", bm.predict(X_val))
print("---------")
print("線形回帰MSE：", f"{mean_squared_error(y_val, lr.predict(X_val)):,.2f}")
print("MSE：", f"{mean_squared_error(y_val, bm.predict(X_val)):,.2f}")

予測値：
 [208154.45406148 147972.99815432 126492.26613158 ... 196301.28219948
  93357.83224498  99185.45986643]
---------
線形回帰MSE： 2,942,066,921.67
MSE： 2,575,081,763.46


一番良いブレンドモデルができている。

In [56]:
#検証4
#前処理をした「線形回帰、決定木」のブレンド

#標準化、対数変換
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
y_train_scaled = np.log(y_train)
y_val_scaled = np.log(y_val)

bm = Blending_model(estimators=[LinearRegression(), DecisionTreeRegressor()])
bm.fit(X_train_scaled, y_train_scaled)
print("予測値：\n", bm.predict(X_val_scaled))
print("---------")
print("線形回帰MSE：", f"{mean_squared_error(y_val, lr.predict(X_val)):,.2f}")
print("MSE：", f"{mean_squared_error(y_val_scaled, bm.predict(X_val_scaled)):,.2f}")

予測値：
 [12.17706851 11.76949193 11.7382464  ... 12.05406506 11.55745489
 11.59022568]
---------
線形回帰MSE： 2,910,319,370.76
MSE： 0.05


/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/itonaoki/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


検証1、3、4で精度が上がっていることを確認

#### 参考サイト

[Blending.1](https://www.quora.com/What-is-blending-in-machine-learning)  
>機械学習におけるブレンドとは何ですか？

>短い答えは次のとおりです。多くの個別のモデルを使用して初期予測を計算し、さらにいくつかの方法で予測を混合して、さらに優れた最終予測を達成する方法。
>
>より正確に言うと、たとえばチームで問題に取り組んだ場合。コンテストの場合、誰もが独自のモデルに取り組んでいます（人々はアイデアについて話し合いますが、誰もが自分のモデルを所有およびコーディングしています）。その後、通常は単純なニューラルネットワークを使用してすべての結果を混合します。入力は問題の説明とさまざまなモデルの結果であり、出力は最終予測です。
>
>時々、単に各モデルの予測の加重平均を使用するだけで十分であり、ニューラルネットワークの必要性を排除します。
>
>時には、さらに複雑な方法が使用されます。詳細については、Rich Caruanaによる「モデルのライブラリからのアンサンブルの選択」を読むことをお勧めします。

[Blending.2](https://mlwave.com/kaggle-ensembling-guide/)
>アンサンブルを行う最も基本的で便利な方法は、Kaggle送信CSVファイルをアンサンブルすることです。これらのメソッドのテストセットの予測のみが必要です。モデルを再トレーニングする必要はありません。これにより、既存のモデル予測をアンサンブルする迅速な方法となり、チーム化するときに理想的です。

[Blending.3](https://www.datarobot.com/wiki/training-validation-holdout/)  


[Blending.4](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/44588)   
>「面白い大会でした。私たちのチームは銀メダルを獲得しました。Kaggleの初心者にとって、それは私たちにとってかなり良い結果でした。
>
>私は議論を見ていて、多くの有用な提案を得ました。しかし、ブレンドとスタックが何であるかはよくわかりません。使用したアンサンブルは、すべてのサンプルで同じ係数を使用した予測の線形結合であり、トレーニングセットを検証セットに分割して係数を選択しました。これはブレンドですか？次に、スタッキングとの違いは何ですか？アンサンブル法を「linear-combination-method」と呼びました。正式名称はありますか？私はOOFと呼ばれるメソッドを見ましたが、それが何であるかわかりません（そしてGoogleで結果を見つけることができません）。また、これに使用するパッケージがあるかと思っていました。私は自分でブレンディングをコーディングしました。
>
>ありがとう！」  


>「おめでとうございます！
>
>私がそれを正しく理解していれば、あなたがしたことは、人々が「ブレンド」によってしばしば意味することであったようにあなたは正しいようです。人々はこれらの用語の多くをかなり大まかに使用しているため、しばしばかなり不明確です。ここに私が標準的な定義として考えるものがあります：
>
>**ブレンド**：  
>トレーニングデータの一部を保持します（たとえば、80/20の分割）。80パーツでベースモデルをトレーニングし、20パーツとテストセットで予測します。機能として20セットの予測を使用してメタラーナーをトレーニングし、最終的な提出予測のテストセットでメタラーナーを実行します。
>
>**スタッキング**：  
>トレーニングデータをフォールドに分割します（たとえば5）。各トレーニングフォールドでベースモデルをトレーニングし、各検証フォールドで予測し、これらの予測を収集します（これがOOF部分の出所です。このようにして、トレーニングデータセット全体に対して行われた予測を収集しますが、「フォールド外」の予測であるため、モデルは、予測したデータに基づいてトレーニングされていませんでした）。次に、これらのOOF予測についてメタラーナーをトレーニングし、最終予測のためにテストセットでメタラーナーを実行します。
>
>この包括的なガイドは、役に立つリソースかもしれません。
>
>再パッケージ化-多くの人々は（単純な）スタッキングを手作業でコーディングしますが、StackNetに興味があるかもしれません。」

---

### 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


[sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。

#### バギングクラスの作成

In [11]:
class Bagging_model():
    """
    ブレンディングのスクラッチ実装

    Parameters
    ----------
    
    """
    def __init__(self, estimator, boot_num):
        self.estimator = estimator
        self.boot_num = boot_num
        
    def _bootstrap(self, X, y):
        """
        N種類のサブセット（ ブートストラップサンプル ）を作り出す
        学習データから重複を許した上でランダムに抜き出すことで、N個の学習データを作成する
        
        Parameters
        ----------
        X : ndarray(n_samples, n_features)
            訓練データの特徴量
        y : ndarray(n_samples, )
            正解データ
        """
        self.boot_Xdic = {}
        self.boot_ydic = {}
        
        #訓練データから重複有りでランダム抽出して、指定個数分を辞書に保存
        for i in range(self.boot_num):
            idx = np.random.choice(X.shape[0], X.shape[0])
            self.boot_Xdic[i] = X[idx]
            self.boot_ydic[i] = y[idx]
        
    def fit(self, X, y):
        """
        N個のサブセットを独立して学習させて、各学習モデルを辞書に保存
        
        Parameters
        ----------
        X : ndarray(n_samples, n_features)
            訓練データの特徴量
        y : ndarray(n_samples, )
            正解データ
        estimators : [model1(), model2(), ...] 
            学習させるモデル
        
        """
        self._bootstrap(X, y) #ブートストラップサンプル作成
        self.estimator_dic = {}
        
        for i in range(self.boot_num):
            model = self.estimator #モデルのインスタンス化
            self.estimator_dic[i] = model.fit(self.boot_Xdic[i], self.boot_ydic[i]) #fitさせたモデルを辞書に保存
        
        pass
    
    def predict(self, X):
        """
        辞書に保存された各学習モデルの予測値を算出後、その平均値を計算
        
        Parameters
        ----------
        X : ndarray(n_samples, n_features)
            検証データの特徴量
        
        Return
        ----------
        y_pred : ndarray(n_samples, )
            検証データの予測値
        """
        predict_list = []
        
        for _, model in self.estimator_dic.items():
            predict_list.append(model.predict(X))
        
        #print("predict_list:\n", predict_list)
        
        y_pred = np.mean(predict_list, axis=0)
        
        return y_pred

In [67]:
#単一な決定木モデルとバギング決定木モデルの精度比較

bag_tree = Bagging_model(estimator=DecisionTreeRegressor(), boot_num=3)
bag_tree.fit(X_train, y_train)
print("予測値：\n", bag_tree.predict(X_val))
print("---------")
print("決定木単一 MSE：", f"{mean_squared_error(y_val, tree.predict(X_val)):,.2f}")
print("決定木バギング MSE：", f"{mean_squared_error(y_val, bag_tree.predict(X_val)):,.2f}")

予測値：
 [278000. 140200. 135000. ... 133900. 118000.  68400.]
---------
決定木単一 MSE： 3,374,967,201.20
決定木バギング MSE： 3,041,810,038.78


バギングをすることで精度が上がった。

In [13]:
bag_tree.estimator_dic

{0: DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
 1: DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
 2: DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
 3: DecisionTreeRegressor(criterio

### 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは$K_0 = 3, M_0 = 2$程度にします。


《学習時》

（ステージ 0）
- 学習データを$K_0$個に分割する。
- 分割した内の$(K_0 -1)$個をまとめて学習用データ、残り1個を推定用データとする組み合わせが$K_0$個作れる。
- あるモデルのインスタンスを$K_0$個用意し、異なる学習用データを使い学習する。
- それぞれの学習済みモデルに対して、使っていない残り1個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
- さらに、異なるモデルのインスタンスも$K_0$個用意し、同様のことを行う。モデルが$M_0$個あれば、$M_0$個のブレンドデータが得られる。

（ステージn）
- ステージ$n -1$のブレンドデータを$M_{n-1}$次元の特徴量を持つ学習用データと考え、$K_n$個に分割する。以下同様である。

（ステージN）＊最後のステージ
- ステージ$n -1$個のブレンドデータを$M_{n-1}$次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》

（ステージ0）
- テストデータを$K_n$×$M_n$個の学習済みモデルに入力し、$K_n$×$M_n$個の推定値を得る。これを$K_0$の軸で平均値を求め$M_0$次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージn）
- ステージ$n -1$で得たブレンドテストを$K_n$×$M_n$個の学習済みモデルに入力し、$K_n$×$M_n$個の推定値を得る。これを$K_n$の軸で平均値を求め$M_0$次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージN）＊最後のステージ
- ステージ$N -1$で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

#### スタッキングクラスの作成

In [39]:
from copy import deepcopy

class Stacking_model():
    """
    スタッキングのスクラッチ実装

    Parameters
    ----------
    
    """
    def __init__(self, estimators, split_size=3):
        self.estimators = estimators
        self.split_size = split_size
        
    def fit(self, X, y):
        self.fit_model_lst = [] #学習済モデル
        pred_list = [] #ブレンドデータの推定値
        y_val_list = []
        
        #分割データで各モデルの学習と推定
        kf = KFold(n_splits=self.split_size, random_state=0, shuffle=True)
        for train_idx, val_idx in kf.split(X):
            X_train_kf, X_val = X[train_idx], X[val_idx]
            y_train_kf, y_val = y[train_idx], y[val_idx]
            for model in deepcopy(self.estimators):
                model_fitted = model.fit(X_train_kf, y_train_kf) #学習
                self.fit_model_lst.append(model_fitted)

                pred = model_fitted.predict(X_val) #ブレンドデータの推定値                 
                pred_list.append(pred)
            y_val_list.extend(y_val) 
        
        blend_data = np.vstack((np.concatenate((pred_list[i::len(self.estimators)])) for i in range(len(self.estimators)))).T
        
        self.final_fitted = LinearRegression().fit(blend_data, np.array(y_val_list).reshape(-1, 1)) #最終モデル
        

    def predict(self, X):
        pred_list = []
        for model_fitted in self.fit_model_lst:
            pred = model_fitted.predict(X)
            pred_list.append(pred)
        
        blend_test = np.vstack(np.mean(np.array(pred_list[i::len(self.estimators)]), axis=0) for i in range(len(self.estimators))).T
        last_pred = self.final_fitted.predict(blend_test)
        return last_pred

In [41]:
#最終モデルは線形回帰

st = Stacking_model(estimators=[LinearRegression(), DecisionTreeRegressor()])
st.fit(X_train, y_train)

print("予測値：\n", st.predict(X_val))
print("---------")
print("線形回帰MSE：", f"{mean_squared_error(y_val, lr.predict(X_val)):,.2f}")
print("スタッキングMSE：", f"{mean_squared_error(y_val, st.predict(X_val)):,.2f}")

予測値：
 [[248541.26697303]
 [153349.80868894]
 [127338.92042433]
 ...
 [217732.65514516]
 [ 70970.34548431]
 [ 92513.48144455]]
---------
線形回帰MSE： 2,942,066,921.67
スタッキングMSE： 2,626,882,871.86


スタッキングにより単一モデルより精度があがることを確認した。